In [1]:
#Arbol elemental con libreria  rpart
#Debe tener instaladas las librerias  data.table  ,  rpart  y  rpart.plot

#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: data.table
Loading required package: rpart
Loading required package: rpart.plot


In [38]:
dataset  <- fread("./datasets/competencia1_2022_FE.csv")


In [39]:
dataset[,clase_binaria:=ifelse(clase_ternaria=='CONTINUA','NO','SI')]

In [40]:
ftable()

ERROR: Error in ftable.default(): nothing to tabulate


In [41]:
dataset<-dataset[,-"clase_ternaria"]

In [42]:
head(dataset[,.(nu=count(numero_de_cliente)),by=clase_binaria])

ERROR: Error in count(numero_de_cliente): no se pudo encontrar la función "count"


In [ ]:
dataset[,.(var=N.), by=clase_binaria]

In [43]:
dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo


In [20]:
#genero el modelo,  aqui se construye el arbol
modelo  <- rpart(formula=   "clase_binaria ~ . ",  #quiero predecir clase_ternaria a partir de el resto de las variables
                 data=      dtrain,  #los datos donde voy a entrenar
                 xval=      0,
                 cp=       -0.3,   #esto significa no limitar la complejidad de los splits
                 minsplit=  0,     #minima cantidad de registros para que se haga el split
                 minbucket= 1,     #tamaño minimo de una hoja
                 maxdepth=  4 )    #profundidad maxima del arbol

In [44]:
#grafico el arbol
#prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0)


#aplico el modelo a los datos nuevos
prediccion  <- predict( object= modelo,
                        newdata= dapply,
                        type = "prob")



In [45]:
#prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
#cada columna es el vector de probabilidades 

#agrego a dapply una columna nueva que es la probabilidad de BAJA+2
dapply[ , prob_baja2 := prediccion[, "SI"] ]


In [46]:

#solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40
dapply[ , Predicted := as.numeric( prob_baja2 > 0.044 ) ]

#genero el archivo para Kaggle
#primero creo la carpeta donde va el experimento
#dir.create( "./exp/" )
#dir.create( "./exp/KA2001" )

fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "Kbinaria.csv",
        sep=  "," )

In [47]:
2/40

[1] 0.05

In [35]:
ftable(dapply[,prob_baja2])

 0.0024390243902439 0.00588235294117647 0.00942126514131898 0.0128767645936665 0.0237830629028673 0.032319391634981 0.0355164548712936 0.0368852459016394 0.0405405405405405 0.0516990291262136 0.0887487875848691 0.139511709018435 0.361370716510903 0.636363636363636      1
                                                                                                                                                                                                                                                                               
             134728                 346                 780              10392               4518               518               2796                236                221               4047               1761              2238               304                11      4